# REST API
In this notebook we will discuss how to use REST API to pull data. We assume the app id is authenticate with secret token.

In [9]:
# install all the azure identity modules
%pip install azure.identity
# install all the azure keyvault modules
%pip install azure-keyvault-secrets

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\huanminw\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\huanminw\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [10]:
from azure.identity import AzureCliCredential, ChainedTokenCredential, DefaultAzureCredential, ManagedIdentityCredential
from azure.keyvault.secrets import SecretClient
from azure.identity import ClientSecretCredential
import requests;

class KeyVault():
    def __init__(self, configuration : dict):
        key_vault = configuration.get('keyvault');
        credential = ChainedTokenCredential(ManagedIdentityCredential(), 
                                            AzureCliCredential(), 
                                            DefaultAzureCredential());
        kv_url = "https://{key_vault}.vault.azure.net";
        kv_url = kv_url.format(key_vault = key_vault);
        self.client = SecretClient(vault_url=kv_url, credential=credential)

    # write a database to database, mode = append or replace
    def get(self, secret_name : str) -> str:
        secret_value = self.client.get_secret(secret_name);
        return secret_value.value;


class RestAPI():
    def __init__(self, configuration : dict):
        app_id = configuration.get('app_id');
        app_secret = configuration.get('app_secret');
        tenant_id = '72f988bf-86f1-41af-91ab-2d7cd011db47';
        credential = ClientSecretCredential(tenant_id, app_id, app_secret);
        self.token = credential.get_token("280b00f4-2c5a-44d8-bab9-6acbbaaa7c78/.default").token;
    
    def call_data_api(self, uri : str) -> dict:
        result = [];
        try:
            headers = {
                "Authorization": "Bearer " + self.token,
                "Content-Type": "application/json"  # Adjust content type as per your API
            }
            response = requests.get(uri, headers=headers);
            if response.status_code == 200:
                result = response.json();
            else:
                print("Error:", response.status_code)
        except requests.exceptions.RequestException as e:
            print("Error:", e);
        return result;

key_vault = KeyVault({'keyvault' : 'COICDIKV'});
app_id = key_vault.get('serviceprincipal');
app_secret = key_vault.get('serviceprincipalkey');
api_client = RestAPI({'app_id' : app_id, 'app_secret': app_secret});
result = api_client.call_data_api("https://api.infra.global.azure.com/pantheon/api/v2/entities/AGI_Region")
print(result)



{'result': [{'regionOid': 'fb184381-2b0c-449f-83c0-3dfa6764322b', 'regionName': 'Japan East', 'geographyOid': '8cc59c09-22b8-4f0e-8e1c-d1ea56c08a6d', 'cloudOid': '45ae26d9-cd6a-44bd-9655-32be2b6c77e7', 'regionCode': 'japaneast', 'state': 'Production SLA', 'availabilityZoneSupport': 'Approved', 'regionType': 'Hub', 'announced': True, 'regionLaunchArchitecture': '1+1', 'zoneLaunchArchitecture': '3 Zones', 'currentArchitecture': '3+1', 'shortId': 'Ty', 'billingName': 'JA East', 'networkZone': 'Zone 2', 'cdnZone': 'Zone 2', 'dataTransferZone': 'Asia', 'afdZone': 'Zone 2', 'metro': 'Tokyo', 'managedAccessDate': '2019-04-18T00:00:00-07:00', 'unrestrictedAccessDate': None, 'timezone': 'Japan Standard Time', 'ownerAlias': 'hesongwang', 'owner': '4d8e5bf6-6d94-40e6-a4db-6ad5a7149256', 'celContactAlias': 'asasaki', 'celContact': 'acd1f54d-fd87-47a6-9f26-5207a7cbe10b', 'celStartDate': '2019-04-01T00:00:00-07:00', 'recoveryRegionOid': None, 'latitude': 35.6869621276855, 'longitude': 139.7494659423